In [1]:
BROKER = "mqtt.eclipseprojects.io"
TOPIC = "/gw01/duese03"

import numpy as np
from bqplot import pyplot as plt, LinearScale
from numpy import arange, pi, sin
import ipywidgets as widgets
from ipywidgets import VBox, HBox
from IPython.display import display, clear_output
import time
import paho.mqtt.client as mqtt

# Given function
def f(x, X=400, x0=0):
    y = abs(sin(2*pi*1/X*(x-x0)))
    y[y<0] = 0
    return y

# Data
x = arange(0, 400.1, 5)
y = f(x)

y0 = 1.4

# Create the figure and scales
fig = plt.figure(title="Function f(x)", animation_duration=0)

x_scale = LinearScale()
y_scale = LinearScale()

# Create the profile plot
profile = plt.plot(x, y, colors=['blue'], labels=['f(x)'], scales={'x': x_scale, 'y': y_scale})
plt.xlabel("x")
plt.ylabel("y")

xs = x[0]
ys = y[0]

# Create the sample vertical line plot
sample = plt.plot([xs, xs], [y0, ys], colors=['red'], labels=['sample'], scales={'x': x_scale, 'y': y_scale})

# Create the lower end symbol (open circle)
lower_end_symbol = plt.plot(x=[xs], y=[ys], marker='circle', colors=['red'])
upper_end_symbol = plt.plot(x=[xs], y=[y0], marker='square', colors=['black'])

# Create the slider
slider = widgets.IntSlider(min=0, max=len(x)-1, value=0, description="Choose xs", orientation="horizontal")

# MQTT setup
def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))

client = mqtt.Client()
client.on_connect = on_connect
client.connect(BROKER, 1883, 60)
client.loop_start()

# Define the update function for the slider
def update_x(change):
    xs = x[change['new']]
    ys = y[change['new']]
    sample.x = [xs, xs]
    sample.y = [y0, ys]
    lower_end_symbol.x = [xs]
    lower_end_symbol.y = [ys]
    upper_end_symbol.x = [xs]
    upper_end_symbol.y = [y0]

    # Publish data over MQTT
    t = time.time()
    topic = TOPIC
    data = "{},{},{}".format(t, xs, ys)
    client.publish(topic, data)

# Link the slider to the update function
slider.observe(update_x, 'value')

# Play function for automatic slider movement
play = widgets.Play(min=0, max=len(x)-1, interval=100, description="Play", step=1, continuous_update=False)

# Update function for the play widget
def update_play(change):
    current_value = change['new']
    max_value = change['owner'].max
    min_value = change['owner'].min

    # Check if reached the limit
    if current_value >= max_value:
        change['owner'].step = -1
    elif current_value <= min_value:
        change['owner'].step = 1

# Link the play widget to the update function
play.observe(update_play, 'value')

# Join the play widget and slider using jslink
widgets.jslink((play, 'value'), (slider, 'value'))

# Arrange the slider and play button side by side
controls = HBox([slider, play])

# Display the plot and controls together using VBox
display(VBox([fig, controls]))
